In [2]:
import csv
import time
import sys
import matplotlib.pyplot as plt
import numpy as np
import math
import os.path
from scipy.stats import multivariate_normal
import ipywidgets as widgets
from IPython.display import display
import pandas as pd


In [36]:
def loadFile(filename):
    '''
    Reads in data file
    '''
    
    
    if 'Barcodes' in filename:
        col_name = ['Subject', 'Barcode']
    elif 'Landmark_Groundtruth' in filename:
        col_name = ['Subject', 'x', 'y', 'x_std', 'y_std']
    elif 'Groundtruth' in filename:
        col_name = ['Time', 'x', 'y', 'orientation']
    elif 'Measurement' in filename:
        col_name = ['Time', 'Subject', 'Range', 'Bearing']
    else:
        col_name = ['Time', 'Forward Velocity', 'Angular Velocity']
        
        
    data = pd.read_csv(filename, names = col_name, comment="#", delimiter = "\t", skipinitialspace = True)
    
    return data

In [38]:
loadFile("./MRCLAM_Dataset1/Landmark_Groundtruth.dat")

,Subject,x,y,x_std,y_std
0,6,5.709283,4.964045,0.000275,0.000415
1,7,5.252926,5.536569,0.000119,0.000354
2,8,3.699877,4.466423,0.000115,0.000212
3,9,2.314463,3.374987,0.000118,0.000121
4,10,0.481799,4.395972,0.000107,0.000185
5,11,3.150720,2.382949,0.000126,0.000138
6,12,4.063288,0.944294,0.000061,0.000134
7,13,2.680344,0.268352,0.000099,0.000095
8,14,0.948285,0.756013,0.000169,0.000151
9,15,3.762902,-2.030915,0.000115,0.000175
